In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, BatchNormalization,Activation
from keras.preprocessing.image import ImageDataGenerator
from keras import backend
from keras.optimizers import RMSprop, Adam
from keras import regularizers
from keras.callbacks import LearningRateScheduler
from keras.models import load_model

from keras.utils import to_categorical

def unpickle(file, train=True):
    import pickle
    with open('./cifar-10-batches-py/'+(('data_batch_'+str(file)) if train else 'test_batch'), 'rb') as fo:
        dict_d = pickle.load(fo,encoding='latin-1')
    return np.reshape(dict_d['data'],(10000,3,32,32)).transpose(0, 2, 3, 1), dict_d['labels']
# def normalization(x):
#     return x/255

training_set = []
training_label = []
for i in range(1,5):
    x,y = unpickle(i)
    training_set.extend(x) 
    training_label.extend(y)

test_data,test_label = unpickle(5)

test_data_norm = np.array(test_data) / 255
test_label_one_hot_encoded = to_categorical(test_label)

training_set_norm = np.array(training_set) / 255
training_label_one_hot_encode = to_categorical(training_label)

batch_size = 128
epochs = 10
IMG_HEIGHT = 32
IMG_WIDTH = 32

labels_name = [
"airplane",
"automobile",
"bird",
"cat",
"deer",
"dog",
"frog",
"horse",
"ship",
"truck"
]

Using TensorFlow backend.


In [2]:
from __future__ import print_function
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
import numpy as np
import os

# Training parameters
batch_size = 32  # orig paper trained all networks with batch_size=128
epochs = 200
data_augmentation = True
num_classes = 10

# Subtracting pixel mean improves accuracy
subtract_pixel_mean = True

# Model parameter
# ----------------------------------------------------------------------------
#           |      | 200-epoch | Orig Paper| 200-epoch | Orig Paper| sec/epoch
# Model     |  n   | ResNet v1 | ResNet v1 | ResNet v2 | ResNet v2 | GTX1080Ti
#           |v1(v2)| %Accuracy | %Accuracy | %Accuracy | %Accuracy | v1 (v2)
# ----------------------------------------------------------------------------
# ResNet20  | 3 (2)| 92.16     | 91.25     | -----     | -----     | 35 (---)
# ResNet32  | 5(NA)| 92.46     | 92.49     | NA        | NA        | 50 ( NA)
# ResNet44  | 7(NA)| 92.50     | 92.83     | NA        | NA        | 70 ( NA)
# ResNet56  | 9 (6)| 92.71     | 93.03     | 93.01     | NA        | 90 (100)
# ResNet110 |18(12)| 92.65     | 93.39+-.16| 93.15     | 93.63     | 165(180)
# ResNet164 |27(18)| -----     | 94.07     | -----     | 94.54     | ---(---)
# ResNet1001| (111)| -----     | 92.39     | -----     | 95.08+-.14| ---(---)
# ---------------------------------------------------------------------------
n = 3

# Model version
# Orig paper: version = 1 (ResNet v1), Improved ResNet: version = 2 (ResNet v2)
version = 2

# Computed depth from supplied model parameter n
if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2

# Model name, depth and version
model_type = 'ResNet%dv%d' % (depth, version)

# Load the CIFAR10 data.
(x_train, y_train), (x_test, y_test) = ((np.array(training_set),np.array(training_label)),(np.array(test_data),np.array(test_label)))

# Input image dimensions.
input_shape = x_train.shape[1:]

# Normalize data.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# If subtract pixel mean is enabled
if subtract_pixel_mean:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('y_train shape:', y_train.shape)

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (40000, 32, 32, 3)
40000 train samples
10000 test samples
y_train shape: (40000,)


In [61]:

def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x


def resnet_v1(input_shape, depth, num_classes=10):
    """ResNet Version 1 Model builder [a]

    Stacks of 2 x (3 x 3) Conv2D-BN-ReLU
    Last ReLU is after the shortcut connection.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filters is
    doubled. Within each stage, the layers have the same number filters and the
    same number of filters.
    Features maps sizes:
    stage 0: 32x32, 16
    stage 1: 16x16, 32
    stage 2:  8x8,  64
    The Number of parameters is approx the same as Table 6 of [a]:
    ResNet20 0.27M
    ResNet32 0.46M
    ResNet44 0.66M
    ResNet56 0.85M
    ResNet110 1.7M

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    # Start model definition.
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)
    # Instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:  # first layer but not first stack
                strides = 2  # downsample
            y = resnet_layer(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            if stack > 0 and res_block == 0:  # first layer but not first stack
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model


def resnet_v2(input_shape, depth, num_classes=10):
    """ResNet Version 2 Model builder [b]

    Stacks of (1 x 1)-(3 x 3)-(1 x 1) BN-ReLU-Conv2D or also known as
    bottleneck layer
    First shortcut connection per layer is 1 x 1 Conv2D.
    Second and onwards shortcut connection is identity.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filter maps is
    doubled. Within each stage, the layers have the same number filters and the
    same filter map sizes.
    Features maps sizes:
    conv1  : 32x32,  16
    stage 0: 32x32,  64
    stage 1: 16x16, 128
    stage 2:  8x8,  256

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    inputs = Input(shape=input_shape)
    # v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])

        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
#     y = Dense(128,activation='softmax' )(y)
#     y = Dense(256,activation='softmax' )(y)
    y = Dense(512,activation='softmax' )(y)
    y = Dense(1024,activation='softmax')(y)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model


if version == 2:
    model = resnet_v2(input_shape=input_shape, depth=depth)
else:
    model = resnet_v1(input_shape=input_shape, depth=depth)

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=lr_schedule(0)),
              metrics=['accuracy'])
model.summary()
print(model_type)

Learning rate:  0.001
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_94 (Conv2D)              (None, 32, 32, 16)   448         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_85 (BatchNo (None, 32, 32, 16)   64          conv2d_94[0][0]                  
__________________________________________________________________________________________________
activation_85 (Activation)      (None, 32, 32, 16)   0           batch_normalization_85[0][0]     
_______________________________________________________________________________________

In [62]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 400:
        lr *= 0.5e-5
    if epoch > 300:
        lr *= 0.5e-5
    if epoch > 180:
        lr *= 0.5e-4
    elif epoch > 160:
        lr *= 1e-4
    elif epoch > 120:
        lr *= 1e-3
    elif epoch > 80:
        lr *= 1e-2
    elif epoch > 20:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [63]:
import os
# Prepare model model saving directory.
save_dir = os.path.join(os.getcwd(), 'saved_models_play')
model_name = 'cifar10_%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler]


In [1]:
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

In [64]:

model = load_model('./saved_models_play/cifar10_ResNet29v2_model.065.h5')
# Run training, with or without data augmentation.
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True,
              callbacks=callbacks)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        # set input mean to 0 over the dataset
        featurewise_center=False,
        # set each sample mean to 0
        samplewise_center=False,
        # divide inputs by std of dataset
        featurewise_std_normalization=False,
        # divide each input by its std
        samplewise_std_normalization=False,
        # apply ZCA whitening
        zca_whitening=False,
        # epsilon for ZCA whitening
        zca_epsilon=1e-06,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=0,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # set range for random shear
        shear_range=0.,
        # set range for random zoom
        zoom_range=0.,
        # set range for random channel shifts
        channel_shift_range=0.,
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        # value used for fill_mode = "constant"
        cval=0.,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False,
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format="channels_last",
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                        steps_per_epoch =training_set_norm.shape[0] // 32,
                        validation_data=(x_test, y_test),
                        epochs=epochs, verbose=1, workers=4,
                        callbacks=callbacks)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Using real-time data augmentation.
Epoch 1/200
Learning rate:  0.001
1250/1250 [==============================] - 52s 42ms/step - loss: 2.2690 - acc: 0.1505 - val_loss: 2.0947 - val_acc: 0.1723

Epoch 00001: val_acc improved from -inf to 0.17230, saving model to C:\Users\zhlow2\Desktop\4487\nf\saved_models_play\cifar10_ResNet29v2_model.001.h5
Epoch 2/200
Learning rate:  0.001
1250/1250 [==============================] - 45s 36ms/step - loss: 2.0291 - acc: 0.1863 - val_loss: 1.9831 - val_acc: 0.1868

Epoch 00002: val_acc improved from 0.17230 to 0.18680, saving model to C:\Users\zhlow2\Desktop\4487\nf\saved_models_play\cifar10_ResNet29v2_model.002.h5
Epoch 3/200
Learning rate:  0.001
1250/1250 [==============================] - 45s 36ms/step - loss: 1.9545 - acc: 0.1961 - val_loss: 1.9110 - val_acc: 0.2016

Epoch 00003: val_acc improved from 0.18680 to 0.20160, saving model to C:\Users\zhlow2\Desktop\4487\nf\saved_models_play\cifar10_ResNet29v2_model.003.h5
Epoch 4/200
Learning rate:  0

1250/1250 [==============================] - 45s 36ms/step - loss: 1.1526 - acc: 0.5681 - val_loss: 1.2095 - val_acc: 0.5366

Epoch 00032: val_acc did not improve from 0.55390
Epoch 33/200
Learning rate:  0.0001
1250/1250 [==============================] - 46s 37ms/step - loss: 1.1478 - acc: 0.5708 - val_loss: 1.1423 - val_acc: 0.5568

Epoch 00033: val_acc improved from 0.55390 to 0.55680, saving model to C:\Users\zhlow2\Desktop\4487\nf\saved_models_play\cifar10_ResNet29v2_model.033.h5
Epoch 34/200
Learning rate:  0.0001
1250/1250 [==============================] - 45s 36ms/step - loss: 1.1352 - acc: 0.5783 - val_loss: 1.1288 - val_acc: 0.5609

Epoch 00034: val_acc improved from 0.55680 to 0.56090, saving model to C:\Users\zhlow2\Desktop\4487\nf\saved_models_play\cifar10_ResNet29v2_model.034.h5
Epoch 35/200
Learning rate:  0.0001
1250/1250 [==============================] - 45s 36ms/step - loss: 1.1299 - acc: 0.5758 - val_loss: 1.1570 - val_acc: 0.5526

Epoch 00035: val_acc did not imp

1250/1250 [==============================] - 45s 36ms/step - loss: 0.9726 - acc: 0.6321 - val_loss: 1.0378 - val_acc: 0.6038

Epoch 00064: val_acc did not improve from 0.61180
Epoch 65/200
Learning rate:  0.0001
1250/1250 [==============================] - 45s 36ms/step - loss: 0.9669 - acc: 0.6350 - val_loss: 1.0053 - val_acc: 0.6146

Epoch 00065: val_acc improved from 0.61180 to 0.61460, saving model to C:\Users\zhlow2\Desktop\4487\nf\saved_models_play\cifar10_ResNet29v2_model.065.h5
Epoch 66/200
Learning rate:  0.0001
 572/1250 [============>.................] - ETA: 23s - loss: 0.9627 - acc: 0.6293

InternalError: GPU sync failed

In [ ]:
pickle.dump(history.history,open('./saved_models_play/historyRES1','wb'))

In [7]:
model = load_model('./saved_models/cifar10_ResNet20v1_model.201.h5')
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 600:
        lr *= 0.5e-5
    elif epoch > 400:
        lr *= 0.5e-4
    elif epoch > 300:
        lr *= 0.5e-3
    elif epoch > 180:
        lr *= 0.7e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

In [8]:
print('Using real-time data augmentation.')
# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
    # set input mean to 0 over the dataset
    featurewise_center=False,
    # set each sample mean to 0
    samplewise_center=False,
    # divide inputs by std of dataset
    featurewise_std_normalization=False,
    # divide each input by its std
    samplewise_std_normalization=False,
    # apply ZCA whitening
    zca_whitening=False,
    # epsilon for ZCA whitening
    zca_epsilon=1e-06,
    # randomly rotate images in the range (deg 0 to 180)
    rotation_range=0,
    # randomly shift images horizontally
    width_shift_range=0.1,
    # randomly shift images vertically
    height_shift_range=0.1,
    # set range for random shear
    shear_range=0.,
    # set range for random zoom
    zoom_range=0.,
    # set range for random channel shifts
    channel_shift_range=0.,
    # set mode for filling points outside the input boundaries
    fill_mode='nearest',
    # value used for fill_mode = "constant"
    cval=0.,
    # randomly flip images
    horizontal_flip=True,
    # randomly flip images
    vertical_flip=False,
    # set rescaling factor (applied before any other transformation)
    rescale=None,
    # set function that will be applied on each input
    preprocessing_function=None,
    # image data format, either "channels_first" or "channels_last"
    data_format="channels_last",
    # fraction of images reserved for validation (strictly between 0 and 1)
    validation_split=0.0)

# Compute quantities required for featurewise normalizatio1
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

history = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                        steps_per_epoch =training_set_norm.shape[0] // 32,
                        validation_data=(x_test, y_test),
                        epochs=epochs+600, verbose=1, workers=4, initial_epoch=201,
                        callbacks=callbacks)

Using real-time data augmentation.
Epoch 202/800
Learning rate:  7e-07
1250/1250 [==============================] - 29s 23ms/step - loss: 0.1801 - acc: 0.9860 - val_loss: 0.4918 - val_acc: 0.9114

Epoch 00202: val_acc improved from -inf to 0.91140, saving model to C:\Users\zhlow2\Desktop\4487\nf\saved_models\cifar10_ResNet20v1_model.202.h5
Epoch 203/800
Learning rate:  7e-07
1250/1250 [==============================] - 26s 21ms/step - loss: 0.1798 - acc: 0.9858 - val_loss: 0.4910 - val_acc: 0.9105

Epoch 00203: val_acc did not improve from 0.91140
Epoch 204/800
Learning rate:  7e-07
1250/1250 [==============================] - 26s 21ms/step - loss: 0.1796 - acc: 0.9859 - val_loss: 0.4926 - val_acc: 0.9102

Epoch 00204: val_acc did not improve from 0.91140
Epoch 205/800
Learning rate:  7e-07
1250/1250 [==============================] - 26s 21ms/step - loss: 0.1790 - acc: 0.9868 - val_loss: 0.4916 - val_acc: 0.9102ss: 0.1790 - acc: 

Epoch 00205: val_acc did not improve from 0.91140
Epoc

1250/1250 [==============================] - 25s 20ms/step - loss: 0.1791 - acc: 0.9864 - val_loss: 0.4905 - val_acc: 0.9114

Epoch 00273: val_acc did not improve from 0.91140
Epoch 274/800
Learning rate:  7e-07
1250/1250 [==============================] - 26s 20ms/step - loss: 0.1783 - acc: 0.9865 - val_loss: 0.4923 - val_acc: 0.9094

Epoch 00274: val_acc did not improve from 0.91140
Epoch 275/800
Learning rate:  7e-07
1250/1250 [==============================] - 25s 20ms/step - loss: 0.1786 - acc: 0.9870 - val_loss: 0.4923 - val_acc: 0.91013s - loss: 0.1782 - acc - ETA: 3s - loss: 0.1782 - 

Epoch 00275: val_acc did not improve from 0.91140
Epoch 276/800
Learning rate:  7e-07
1250/1250 [==============================] - 25s 20ms/step - loss: 0.1781 - acc: 0.9865 - val_loss: 0.4912 - val_acc: 0.9105

Epoch 00276: val_acc did not improve from 0.91140
Epoch 277/800
Learning rate:  7e-07
1250/1250 [==============================] - 25s 20ms/step - loss: 0.1774 - acc: 0.9868 - val_loss: 0

1250/1250 [==============================] - 25s 20ms/step - loss: 0.1762 - acc: 0.9875 - val_loss: 0.4926 - val_acc: 0.9095

Epoch 00346: val_acc did not improve from 0.91140
Epoch 347/800
Learning rate:  5e-07
1250/1250 [==============================] - 25s 20ms/step - loss: 0.1773 - acc: 0.9868 - val_loss: 0.4929 - val_acc: 0.91011776 - - ETA: 1s - loss: 0.17 - ETA: 0s - loss: 0.1776 - acc: 0.

Epoch 00347: val_acc did not improve from 0.91140
Epoch 348/800
Learning rate:  5e-07
1250/1250 [==============================] - 26s 20ms/step - loss: 0.1777 - acc: 0.9858 - val_loss: 0.4918 - val_acc: 0.9097-  - ETA: 0s - loss: 0.1777 - acc: 

Epoch 00348: val_acc did not improve from 0.91140
Epoch 349/800
Learning rate:  5e-07
1250/1250 [==============================] - 26s 20ms/step - loss: 0.1756 - acc: 0.9871 - val_loss: 0.4915 - val_acc: 0.9098

Epoch 00349: val_acc did not improve from 0.91140
Epoch 350/800
Learning rate:  5e-07
1250/1250 [==============================] - 25s 20ms

1250/1250 [==============================] - 26s 21ms/step - loss: 0.1766 - acc: 0.9862 - val_loss: 0.4917 - val_acc: 0.90970.98 - ETA: 7s - los - ETA: 1s

Epoch 00418: val_acc did not improve from 0.91140
Epoch 419/800
Learning rate:  5.0000000000000004e-08
1250/1250 [==============================] - 26s 21ms/step - loss: 0.1759 - acc: 0.9870 - val_loss: 0.4926 - val_acc: 0.9096

Epoch 00419: val_acc did not improve from 0.91140
Epoch 420/800
Learning rate:  5.0000000000000004e-08
1250/1250 [==============================] - 26s 21ms/step - loss: 0.1740 - acc: 0.9879 - val_loss: 0.4917 - val_acc: 0.9100

Epoch 00420: val_acc did not improve from 0.91140
Epoch 421/800
Learning rate:  5.0000000000000004e-08
1250/1250 [==============================] - 26s 21ms/step - loss: 0.1761 - acc: 0.9871 - val_loss: 0.4937 - val_acc: 0.9102

Epoch 00421: val_acc did not improve from 0.91140
Epoch 422/800
Learning rate:  5.0000000000000004e-08
1250/1250 [==============================] - 26s 21ms/

1250/1250 [==============================] - 26s 21ms/step - loss: 0.1750 - acc: 0.9874 - val_loss: 0.4917 - val_acc: 0.9101

Epoch 00488: val_acc did not improve from 0.91140
Epoch 489/800
Learning rate:  5.0000000000000004e-08
1250/1250 [==============================] - 26s 21ms/step - loss: 0.1740 - acc: 0.9873 - val_loss: 0.4927 - val_acc: 0.9103

Epoch 00489: val_acc did not improve from 0.91140
Epoch 490/800
Learning rate:  5.0000000000000004e-08
1250/1250 [==============================] - 26s 21ms/step - loss: 0.1754 - acc: 0.9873 - val_loss: 0.4908 - val_acc: 0.9109

Epoch 00490: val_acc did not improve from 0.91140
Epoch 491/800
Learning rate:  5.0000000000000004e-08
1250/1250 [==============================] - 26s 21ms/step - loss: 0.1746 - acc: 0.9875 - val_loss: 0.4941 - val_acc: 0.9098

Epoch 00491: val_acc did not improve from 0.91140
Epoch 492/800
Learning rate:  5.0000000000000004e-08
1250/1250 [==============================] - 26s 21ms/step - loss: 0.1752 - acc: 0.9

KeyboardInterrupt: 

In [12]:
model.save('./modelRES1_511.h5')

In [11]:
import pickle
pickle.dump(history.history,open('historyRES1_2','wb'))

NameError: name 'history' is not defined

In [9]:
pp = np.load('./test_data/y_test.npy')
pp_norm = pp/255
test_res = []
for i in model.predict(pp_norm):
    test_res.append(np.argmax(i))
# plotImages(pp,test_res)
# plotImages(pp[2400:],test_res[2400:])
# plotImages(pp[2400*2:],test_res[2400*2:])
# plotImages(pp[2400*3:],test_res[2400*3:])
# # summarize_diagnostics(history41)
# summarize_diagnostics(history42)
# summarize_diagnostics(history43)

df = pd.DataFrame({'Category':test_res})
df['Index'] = df.index
df = df[['Index','Category']]
df.to_csv('./test_resRES1_2.csv',index=False)